## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
try:
    sw = stopwords.words("english")
except:
    from nltk import download as nltkdl
    nltkdl('stopwords')
    sw =  stopwords.words("english")

In [17]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [18]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions
                            ''')

for row in query_results :
    text = row[0]
    party = row[1]
    #cleaning text
    text = text.lower()
    text = text.replace('\n', ' ')
    for punct in punctuation:
        text = text.replace(punct, '')
    #cleaning text
    text = text.split()
    # store the results in convention_data 
    convention_dict = {'party': party, 'tokens': text}
    convention_data.append(convention_dict)

Let's look at some random entries and see if they look right. 

In [19]:
random.choices(convention_data,k=10)

[{'party': 'Democratic', 'tokens': ['north', 'dakota']},
 {'party': 'Democratic',
  'tokens': ['the',
   'child',
   'who',
   'knows',
   'that',
   'black',
   'lives',
   'matter',
   'she',
   'is',
   'our',
   'warrior',
   'on',
   'the',
   'front',
   'lines',
   'challenging',
   'authority',
   'to',
   'make',
   'the',
   'world',
   'safe',
   'ms']},
 {'party': 'Republican',
  'tokens': ['thank',
   'you',
   'for',
   'being',
   'a',
   'hero',
   'to',
   'everybody',
   'in',
   'the',
   'country']},
 {'party': 'Democratic',
  'tokens': ['i',
   'must',
   'interject',
   'one',
   'thing',
   'at',
   'the',
   'very',
   'end',
   'of',
   'his',
   'chat',
   'with',
   'me',
   'he',
   'said',
   '“could',
   'you',
   'do',
   'me',
   'a',
   'favor”',
   'i',
   'said',
   '“sure',
   'what',
   'is',
   'it”',
   'he',
   'said',
   '“tell',
   'me',
   'how',
   'i',
   'can',
   'get',
   'your',
   'grandson’s',
   'hair”',
   'he',
   'was',
   'good',


If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [51]:
word_cutoff = 5

#tokens = [w for t, p in convention_data for w in t.split()]
tokens = []
for row in convention_data:
    tokens += row['tokens']
    
word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff and word not in stopwords.words('english'):
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [23]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    text_token = text.split()
    for feature_word in fw:
        if feature_word in text_token:
            ret_dict[feature_word] = True
            
    return(ret_dict)

In [25]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [34]:
#featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

featuresets = []

for row in convention_data:
    party = row['party']
    features = conv_features(' '.join(row['tokens']), feature_words)
    feature_sets =  (features, party)
    featuresets.append(feature_sets)

In [35]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [36]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [37]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations
My observation from this classifier is, Republicans use the words China, Enforcement and Destroy a lot more than Democratic.

Whereas Democratic used the words Votes and Climate a lot more than Republicans. We also notice Republican has a lot more top words than democratic.




## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [38]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [39]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [40]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
for row in results:
    candidate = row[0]
    party = row[1]
    tweet = row[2].decode('utf-8')
    
    #applying punctuation function on tweets
    for punct in punctuation:
        tweet = tweet.replace(punct, '')
        
    #replacing next line with empty space
    tweet = tweet.replace('\n', ' ')

    #lowering all tweet words
    tweet = tweet.lower()
    
    #splitting tweets into tokens
    tweet = tweet.split()
    
    tweet_tokens = []
    for twee in tweet:
        if twee not in stopwords.words('english'):
            tweet_tokens.append(twee)
            
    dict_tweet = {'candidate': candidate, 'party': party, 'tweet': tweet_tokens}
    tweet_data.append(dict_tweet)
    
# Note that this may take a bit of time, since we have a lot of tweets.


In [52]:
word_cutoff = 5

#tokens = [w for t, p in convention_data for w in t.split()]
tokens = []
for row in tweet_data:
    tokens += row['tweet']
    
word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff and word not in stopwords.words('english'):
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 49945 as features in the model.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [42]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [43]:

for row in tweet_data_sample :
    # Fill in the right-hand side above with code that estimates the actual party
    party = row['party']
    tweet = ' '.join(row['tweet'])
    features = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(features)
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [46]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet = ' '.join(tp['tweet'])
    party = tp['party']
    #tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    features = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(features)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [47]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3727, 'Democratic': 645}),
             'Democratic': defaultdict(int,
                         {'Republican': 4750, 'Democratic': 880})})

In [50]:
republic_tp = 3727/(3727+645)
democratic_tp = 4750/(4750+880)

republic_fn = 645/(3727+645)
democratic_fn = 880/(4750+880)

print('True Positive Rate for Republican is', republic_tp)
print('False Negative Rate for Democratic is', democratic_tp)
print('True Positive Rate for Republican is', republic_fn)
print('False Negative Rate for Democratic is', democratic_fn)

True Positive Rate for Republican is 0.8524702653247942
False Negative Rate for Democratic is 0.8436944937833037
True Positive Rate for Republican is 0.14752973467520586
False Negative Rate for Democratic is 0.15630550621669628


### Reflections

Looking at our results, We had 3727 republicans that was correctly identified as republican and 645 that was incorrectly identified as democratic.

As for democratic, 4750 was accurately identified as democratic and 880 was incorrectly identified as democratic. 

Overall, this is a great model but can be better with more data and a better approach.